In [ ]:
!pip install --upgrade langchain openai  -q
!pip install sentence_transformers -q
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q
!apt-get install poppler-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is n

In [ ]:
# create a path
import os.path
from os import path

if path.exists('/content/data') == False:
  os.mkdir('/content/data')

In [ ]:
# upload file
%cd /content/data
from google.colab import files
while True:

  uploaded = files.upload()

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  name = int(input ("Press 1 to continue").strip());
  if name != 1:
    break

/content/data


Saving LearnJava.pdf to LearnJava.pdf
User uploaded file "LearnJava.pdf" with length 6039939 bytes
Press 1 to continue1


Saving Data_Structures_and_Algorithms_in_Java_S.pdf to Data_Structures_and_Algorithms_in_Java_S.pdf
User uploaded file "Data_Structures_and_Algorithms_in_Java_S.pdf" with length 4186858 bytes
Press 1 to continue0


https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/directory_loader.html

In [ ]:
from langchain.document_loaders import DirectoryLoader

directory = '/content/data'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


2

https://python.langchain.com/en/latest/modules/indexes/text_splitters/getting_started.html

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=2000,chunk_overlap=500):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

1586


In [ ]:
print(docs[7].page_content)

The Array Workshop Applet ......................................................................33 Insertion ...........................................................................................35 Searching ..........................................................................................36 Deletion ............................................................................................36 The Duplicates Issue .........................................................................37 Not Too Swift ....................................................................................39 The Basics of Arrays in Java .......................................................................39 Creating an Array .............................................................................40 Accessing Array Elements .................................................................40 Initialization ...................................................................................

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

384

In [ ]:
!pip install -qU pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 4.4 MB/s eta 0:00:00


https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/pinecone.html

In [ ]:
from pinecone import Pinecone, PodSpec
from langchain.vectorstores import Pinecone as Pn

pc = Pinecone(api_key="445535a3-d2a3-4ae1-9037-7322276f9a81")
index_name = "langchain-chatbot-pdf-demo"
spec = PodSpec(environment="gcp-starter")
# clean all indexes in project
for data in pc.list_indexes().names():
  pc.delete_index(data)

pc.create_index(
        index_name,
        dimension=len(query_result),  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )

# connect to new create index
index = pc.Index(index_name)
index.describe_index_stats()

text_field = "text"
vectorstore = Pn(
    index, embeddings.embed_query, text_field
)
# Batch insert the chunks into the vector store
batch_size = 100  # Define your preferred batch size
for i in range(0, len(docs), batch_size):
    chunk_batch = docs[i:i + batch_size]
    vectorstore.add_documents(chunk_batch)

print("Ok")

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


Ok


In [ ]:
query = "What is data structure stack ?"

score = False
# switch back to normal index for langchain
index = pc.Index(index_name)

if score:
  similar_docs = vectorstore.similarity_search_with_score(query,k=5)
else:
  similar_docs = vectorstore.similarity_search(query,k=5)
print(similar_docs)

[Document(page_content='113\n\nStacks and Queues\n\nIn this chapter we’ll examine three data storage structures:\n\nthe stack, the queue, and the priority queue. We’ll begin by discussing how these structures differ from arrays; then we’ll examine each one in turn. In the last section, we’ll look at an operation in which the stack plays a significant role: parsing arithmetic expressions.\n\nA Different Kind of Structure\n\nThere are significant differences between the data struc- tures and algorithms we’ve seen in previous chapters and those we’ll look at now. We’ll discuss three of these differ- ences before we examine the new structures in detail.\n\nProgrammer’s Tools\n\nArrays—the data storage structure we’ve been examining thus far—as well as many other structures we’ll encounter later in this book (linked lists, trees, and so on) are appro- priate for the kind of data you might find in a database application. They’re typically used for personnel records, inventories, financial da

In [ ]:
from google.colab import userdata
userdata.